# La modélisation - essai 2 - un peu de machine learning?  
## Cette partie n'a pas abouti malgré nos essais. 

L'objectif de cette partie sera de proposer un algorithme de recommendation de films en prenant en compte leur proximité relative. On veut travailler sur les variables comme le cynopsis pour analyser la proximité sémantique des mots, et utiliser des procédés de machine learning et de la cross-validation pour faire un algorithme qui pourrait chosiir les poids relatifs de chaque variable suite à un stade d'entrainement. Cependant, cette partie n'a pas été réussie, contrairement à la modélisation plus simple que nous avons réalisé en premier. En effet, il semble y avoir un problème dans notre modèle et surtout lors de la récupération du nom du film. N'ayant pas encore étudié les procédés de Machine Learning en cours, et surtout non supervisé, il nous a été difficile de finaliser cette partie. Nous souhaitons tout de même laisser notre tenative ici.

In [17]:
#imports pour nos besoin (notamment pour ajouter une étape de validation) 
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [20]:
def fonction_charge(path) :
    df = pd.read_csv(path)
    df_f = df.drop_duplicates()
    df_f['release_date'] = pd.to_datetime(df_f['release_date'])
    # De la même façon on convertit la colonne 'budget' qui est composée de chaînes de caractères et non de valeurs numériques
    df_f['budget'] = pd.to_numeric(df_f['budget'], errors='coerce')
    df_f['genres_list'] = df_f['genres_list'].apply(ast.literal_eval)
    df_f['countries_prod'] = df_f['countries_prod'].apply(ast.literal_eval)
    df_f['languages_list'] = df_f['languages_list'].apply(ast.literal_eval)
    df_f['prod_companies'] = df_f['prod_companies'].apply(ast.literal_eval)
    return df_f 

In [19]:
# Supposons que vous avez un DataFrame appelé df avec les colonnes appropriées
# Remplacez les données fictives par vos données réelles
df1 = fonction_charge('../cleaning_data/Final_database.csv')
df1.columns

/tmp/ipykernel_6273/49747245.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['release_date'] = pd.to_datetime(df_f['release_date'])
/tmp/ipykernel_6273/49747245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['budget'] = pd.to_numeric(df_f['budget'], errors='coerce')
/tmp/ipykernel_6273/49747245.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Index(['adult', 'budget', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'release_date', 'revenue',
       'runtime', 'tagline', 'title', 'vote_average', 'vote_count',
       'directors', 'collection_name', 'genres_list', 'countries_prod',
       'languages_list', 'prod_companies', 'Award_mains', 'Mains_cat',
       'Award_fest', 'Fest_cat'],
      dtype='object')

In [27]:

# Séparation des données en ensembles d'entraînement et de test
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)

# Entraînement sur l'ensemble d'entraînement
# (placez cette partie dans une boucle si vous voulez essayer différentes configurations de modèles)
for film_reference in train_df.index:
    # Extraction des informations
    realisateurs = train_df["directors"]
    genres = train_df["genres_list"].apply(lambda x: " ".join(x))
    notes = train_df["vote_average"]
    overviews = train_df["overview"]
    budgets = train_df["budget"]
    realisateurs = realisateurs.fillna('')
    overviews = overviews.fillna('')
    # Normalisation des notes et des budgets
    scaler = MinMaxScaler()
    notes_normalisees = scaler.fit_transform(np.array(notes).reshape(-1, 1))
    budgets_normalises = scaler.fit_transform(np.array(budgets).reshape(-1, 1))
    realisateurs_vectorises = CountVectorizer().fit_transform(realisateurs)
    genres_vectorises = CountVectorizer().fit_transform(genres)
    overviews_vectorises = CountVectorizer().fit_transform(overviews)
 
    # Calcul des similarités entre les films en fonction des différentes caractéristiques
    similarity_realisateurs = realisateurs_vectorises.dot(realisateurs_vectorises.T).toarray()
    similarity_genres = genres_vectorises.dot(genres_vectorises.T).toarray()
    similarity_overviews = overviews_vectorises.dot(overviews_vectorises.T).toarray()
    similarity_notes = np.dot(notes_normalisees, notes_normalisees.T)
    similarity_budgets = np.dot(budgets_normalises, budgets_normalises.T)
    
    # Combinaison des similarités avec des poids - pour l'instant arbitraires
    weight_realisateurs = 0.2
    weight_genres = 0.2
    weight_overviews = 0.2
    weight_notes = 0.2
    weight_budgets = 0.2

    similarity_totale = (
        weight_realisateurs * similarity_realisateurs +
        weight_genres * similarity_genres +
        weight_overviews * similarity_overviews +
        weight_notes * similarity_notes +
        weight_budgets * similarity_budgets
    )
    print(similarity_totale)
    # Recommandation pour le film de référence
    if film_reference < len(similarity_totale):
        films_recommandes = np.argsort(similarity_totale[film_reference])[::-1]

        # Affichage des 5 films les plus proches
        print(f"Les 3 films les plus proches de {df1['title'][film_reference]} :")
        for i, film_index in enumerate(films_recommandes[:3]):
            if film_index != film_reference:
                print(f"Recommandation {i + 1}: {df1['title'][film_index]}")
    else:
        print(f"Film de référence {film_reference} hors des limites de l'ensemble de données.")

[[11.63068     0.9300464   4.506227   ...  2.1277496   1.9341846
   6.5192356 ]
 [ 0.9300464  17.43080476  7.10596196 ...  1.32727604  1.93353401
   7.71935193]
 [ 4.506227    7.10596196 42.98639468 ...  6.10387119  5.30907582
  16.09704909]
 ...
 [ 2.1277496   1.32727604  6.10387119 ... 16.22490022  2.93117561
   8.11663475]
 [ 1.9341846   1.93353401  5.30907582 ...  2.93117561 11.23778319
   4.52243328]
 [ 6.5192356   7.71935193 16.09704909 ...  8.11663475  4.52243328
  44.00914067]]
Les 3 films les plus proches de The Mummy Resurrection :
Recommandation 1: Jack Reacher
Recommandation 2: Home on the Range
Recommandation 3: The Key Game
[[11.63068     0.9300464   4.506227   ...  2.1277496   1.9341846
   6.5192356 ]
 [ 0.9300464  17.43080476  7.10596196 ...  1.32727604  1.93353401
   7.71935193]
 [ 4.506227    7.10596196 42.98639468 ...  6.10387119  5.30907582
  16.09704909]
 ...
 [ 2.1277496   1.32727604  6.10387119 ... 16.22490022  2.93117561
   8.11663475]
 [ 1.9341846   1.93353401 

KeyError: 4230